<a href="https://colab.research.google.com/github/Mahsa33/Body-Shape-Classification/blob/main/Grounded_SAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cloning the Repository


In [ ]:
%cd /content
!git clone https://github.com/souvik0306/Grounded-Segment-Anything

/content
Cloning into 'Grounded-Segment-Anything'...
remote: Enumerating objects: 1240, done.
remote: Counting objects: 100% (1240/1240), done.
remote: Compressing objects: 100% (708/708), done.
remote: Total 1240 (delta 529), reused 1164 (delta 521), pack-reused 0
Receiving objects: 100% (1240/1240), 120.42 MiB | 38.40 MiB/s, done.
Resolving deltas: 100% (529/529), done.


In [ ]:
import os

folder_path = 'Grounded-Segment-Anything'
init_file_path = os.path.join(folder_path, '__init__.py')

# Create the __init__.py file
with open(init_file_path, 'w') as f:
    pass

#Installing Dependencies

In [ ]:
%cd /content/Grounded-Segment-Anything
!pip install -q -r requirements.txt
%cd /content/Grounded-Segment-Anything/GroundingDINO
!pip install -q .
%cd /content/Grounded-Segment-Anything/segment_anything
!pip install -q .
%cd /content/Grounded-Segment-Anything

/content/Grounded-Segment-Anything
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 989.1/989.1 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 30.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backen

#Importing Libraries

In [ ]:
import os, sys

if not os.path.exists('Extracted'):
    os.makedirs('Extracted')

if not os.path.exists('Test Folder'):
    os.makedirs('Test Folder')

if not os.path.exists('Final Result'):
    os.makedirs('Final Result')

sys.path.append(os.path.join(os.getcwd(), "GroundingDINO"))

import argparse
import copy

from IPython.display import display
from PIL import Image, ImageDraw, ImageFont
from torchvision.ops import box_convert

# Grounding DINO
import GroundingDINO.groundingdino.datasets.transforms as T
from GroundingDINO.groundingdino.models import build_model
from GroundingDINO.groundingdino.util import box_ops
from GroundingDINO.groundingdino.util.slconfig import SLConfig
from GroundingDINO.groundingdino.util.utils import clean_state_dict, get_phrases_from_posmap
from GroundingDINO.groundingdino.util.inference import annotate, load_image, predict

import supervision as sv

# segment anything
from segment_anything import build_sam, SamPredictor
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

# diffusers
import PIL
import requests
import torch
from io import BytesIO
from diffusers import StableDiffusionInpaintPipeline


from huggingface_hub import hf_hub_download

#Checking for GPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def load_model_hf(repo_id, filename, ckpt_config_filename, device='cpu'):
    cache_config_file = hf_hub_download(repo_id=repo_id, filename=ckpt_config_filename)

    args = SLConfig.fromfile(cache_config_file)
    args.device = device
    model = build_model(args)

    cache_file = hf_hub_download(repo_id=repo_id, filename=filename)
    checkpoint = torch.load(cache_file, map_location=device)
    log = model.load_state_dict(clean_state_dict(checkpoint['model']), strict=False)
    print("Model loaded from {} \n => {}".format(cache_file, log))
    _ = model.eval()
    return model

#Loading Grounding Dino for Zero Shot Object Detection

---



In [ ]:
ckpt_repo_id = "ShilongLiu/GroundingDINO"
ckpt_filenmae = "groundingdino_swinb_cogcoor.pth"
ckpt_config_filename = "GroundingDINO_SwinB.cfg.py"


groundingdino_model = load_model_hf(ckpt_repo_id, ckpt_filenmae, ckpt_config_filename, device)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Model loaded from /root/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


#Installing Weights for SAM

In [ ]:
! wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

sam_checkpoint = 'sam_vit_h_4b8939.pth'

sam_predictor = SamPredictor(build_sam(checkpoint=sam_checkpoint).to(device))

--2023-09-04 04:52:28--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.162.163.11, 3.162.163.19, 3.162.163.34, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.162.163.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   161MB/s    in 19s     

2023-09-04 04:52:47 (132 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Defining Object to Segment

In [ ]:
# detect object using grounding DINO
def detect(image, text_prompt, model, image_source, box_threshold = 0.3, text_threshold = 0.25 ):
  boxes, logits, phrases = predict(
      model=model,
      image=image,
      caption=text_prompt,
      box_threshold=box_threshold,
      text_threshold=text_threshold
  )

  annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
  annotated_frame = annotated_frame[...,::-1] # BGR to RGB
  return annotated_frame, boxes

In [ ]:
object_to_segment = "Person"

In [ ]:
def segment(image, sam_model, boxes):
  sam_model.set_image(image)
  H, W, _ = image.shape
  boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.Tensor([W, H, W, H])

  transformed_boxes = sam_model.transform.apply_boxes_torch(boxes_xyxy.to(device), image.shape[:2])
  masks, _, _ = sam_model.predict_torch(
      point_coords = None,
      point_labels = None,
      boxes = transformed_boxes,
      multimask_output = False,
      )
  return masks.cpu()


def draw_mask(mask, image, random_color=True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.8])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)

    annotated_frame_pil = Image.fromarray(image).convert("RGBA")
    mask_image_pil = Image.fromarray((mask_image.cpu().numpy() * 255).astype(np.uint8)).convert("RGBA")

    return np.array(Image.alpha_composite(annotated_frame_pil, mask_image_pil))

In [ ]:
import groundingdino.datasets.transforms as T
from typing import Tuple, List
def load_image(image_path: str) -> Tuple[np.array, torch.Tensor]:
    transform = T.Compose(
        [
            T.RandomResize([800], max_size=1333),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )
    image_source = Image.open(image_path).convert("RGB")
    image = np.asarray(image_source)
    image_transformed, _ = transform(image_source, None)
    return image, image_transformed

#Loading Images



In [ ]:
# save numpy array as npy file
from numpy import asarray
from numpy import save

# load numpy array from npy file
from numpy import load

In [ ]:
def seg_img(name_lst, img_lst):
  count = 0
  directory = "/content/drive/MyDrive/thesis_datasets/seg_final_data/" + name_lst
  if not os.path.exists(directory):
    os.makedirs(directory)

  segmented_masks_dict = {}

  for img_path in img_lst:
    pth , img_name = os.path.split(img_path)
    img_name = img_name[:-4]

    image_source, image = load_image(img_path)
    Image.fromarray(image_source)

    annotated_frame, detected_boxes = detect(image, text_prompt=object_to_segment, model=groundingdino_model, image_source=image_source)
    segmented_frame_masks = segment(image_source, sam_predictor, boxes=detected_boxes)
    segmented_masks_dict[img_name] = segmented_frame_masks
    # save to npy file
    save(directory + 'segmented_masks.npy', segmented_masks_dict)
    annotated_frame_with_mask = draw_mask(segmented_frame_masks[0][0], image_source)
    Image.fromarray(annotated_frame_with_mask).save(directory + '/' + img_name + '.png')
    count += 1
    print(count)

In [ ]:
!pip install patool
import patoolib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 1.6 MB/s eta 0:00:00


In [ ]:
zip_path = '/content/drive/MyDrive/thesis_datasets/body_img_rar'
unzip_path = '/content/drive/MyDrive/thesis_datasets/body_img'

for fol in os.listdir(zip_path):
  cleb_path = ''.join([zip_path, '/', fol])
  patoolib.extract_archive(cleb_path, outdir = unzip_path)

In [ ]:
body_shape = {'Triangle': [], 'Apple': [], 'Rectangle': [], 'InvertedTriangle': [], 'Hourglass': []}

for shape in body_shape.keys():
  shape_path = ''.join([unzip_path, "/", shape])
  body_shape[shape] = [''.join([shape_path, '/', img]) for img in os.listdir(shape_path)]

Triangle, Apple, Rectangle, InvertedTriangle, Hourglass = body_shape.values()

In [ ]:
seg_img('Triangle',Triangle)

In [ ]:
print('number of triangle segmentation images',len(os.listdir('/content/drive/MyDrive/thesis_datasets/seg_final_data/Triangle')))
print('number of triangle images',len(os.listdir('/content/drive/MyDrive/thesis_datasets/body_img/Triangle')))

In [ ]:
seg_img('Apple',Apple)

In [ ]:
print('number of apple segmentation images',len(os.listdir('/content/drive/MyDrive/thesis_datasets/seg_final_data/Apple')))
print('number of apple images',len(os.listdir('/content/drive/MyDrive/thesis_datasets/body_img/Apple')))

In [ ]:
seg_img('InvertedTriangle',InvertedTriangle)

In [ ]:
print('number of InvertedTriangle segmenatation images',len(os.listdir('/content/drive/MyDrive/thesis_datasets/seg_final_data/InvertedTriangle')))
print('number of InvertedTriangle images',len(os.listdir('/content/drive/MyDrive/thesis_datasets/body_img/InvertedTriangle')))

In [ ]:
seg_img('Rectangle',Rectangle)

In [ ]:
print('number of Rectangle segmenatation images',len(os.listdir('/content/drive/MyDrive/thesis_datasets/seg_final_data/Rectangle')))
print('number of Rectangle images',len(os.listdir('/content/drive/MyDrive/thesis_datasets/body_img/Rectangle')))

In [ ]:
seg_img('Hourglass',Hourglass)

In [ ]:
print('number of Hourglass segmenatation images',len(os.listdir('/content/drive/MyDrive/thesis_datasets/seg_final_data/Hourglass')))
print('number of Hourglass images',len(os.listdir('/content/drive/MyDrive/thesis_datasets/body_img/Hourglass')))

In [ ]:
# use this class if you want remove and resize corrupted segmentation
class LoadRemveResiz:

  def __init__(self,npy_file_path, imgs_path, batch_size, size):
    self.tensor_dict = np.load(npy_file_path, allow_pickle=True).item()
    self.imgs_path = imgs_path
    self.batch_size = batch_size
    self.size = size

  def identify_damaged_img(self):
    self.damaged_img = []
    self.path = os.listdir(self.imgs_path)
    self.img_path = []
    self.tensor_key = []

    for img in self.path:
      self.img_path.append(img[:-4])


    for key in self.tensor_dict.keys():
        self.tensor_key.append(key)


    for img_name in self.tensor_key:
      if img_name not in self.img_path:
        self.damaged_img.append(img_name)
    return self.damaged_img

  def remove_damaged_img(self, img_lst):
    for key in img_lst:
      self.tensor_dict.pop(key, None)

    return self.tensor_dict

  def resize_tensors(self):
    resized_tensor_dict = {}
    segment_all = {}
    keys = list(self.tensor_dict.keys())
    num_tensors = len(keys)
    num_batches = (num_tensors + self.batch_size - 1) // self.batch_size

    for i in range(num_batches):
        start_idx = i * self.batch_size
        end_idx = min(start_idx + self.batch_size, num_tensors)

        batch_keys = keys[start_idx:end_idx]
        batch_tensors = [self.tensor_dict[key] for key in batch_keys]

        resized_batch = []
        for tensor in batch_tensors:
            resized_tensor = F.interpolate(tensor.float(), size=self.size, mode='bilinear', align_corners=False)
            resized_batch.append(resized_tensor)

        for j, key in enumerate(batch_keys):
            resized_tensor_dict[key] = resized_batch[j]


        segment_all.update(resized_tensor_dict)

    return resized_tensor_dict


  def resize_images(self):
      damaged_images = self.identify_damaged_img()
      self.remove_damaged_img(damaged_images)
      resized_tensors = self.resize_tensors()
      return resized_tensors

In [ ]:
loader = LoadRemveResiz('/content/drive/MyDrive/thesis_datasets/seg_final_data/trisegmented_masks.npy', '/content/drive/MyDrive/thesis_datasets/seg_final_data/tri',batch_size=150, size=(224, 224))
resized_images = loader.resize_images()

In [ ]:
np.save('/content/drive/MyDrive/thesis_datasets/seg_resize/resized_t.npy', resized_images)